In [54]:
import pandas as pd
import geopandas as gpd

from shared_utils import rt_utils, catalog_utils, rt_dates, webmap_utils

from calitp_data_analysis import get_fs, geography_utils
from calitp_data_analysis.gcs_geopandas import GCSGeoPandas
gcsgp = GCSGeoPandas()
from segment_speed_utils import helpers, time_series_utils, gtfs_schedule_wrangling, corridor_analysis
from segment_speed_utils.project_vars import SCHED_GCS, SEGMENT_GCS, GTFS_DATA_DICT
import numpy as np
import datetime as dt

ADHOC_PATH = "gs://calitp-analytics-data/data-analyses/rt_segment_speeds/adhoc/"

# Run SCCP/LPP Level of Transit Delay Metric using newer corridor tools

> The speed-based metric for the corridor provided is 2,160 minutes. This is the sum of delays for each trip traversing the corridor as compared to a reference speed of 16 miles per hour. To further explain, we took each corridor trip that we have data for and calculated the hypothetical time it would take for that trip to traverse the corridor at a speed of 16 mph. The difference between the actual time it took for the trip to traverse the corridor and that hypothetical time is the speed-based delay for that trip, and we summed those delays to create the metric for a day (October 15, 2025).
>
> This metric is intended to provide a more consistent basis for comparison independent of scheduling practices.
>
> To calculate which routes we consider to be along the corridor, we first filtered to routes stopping within the corridor. We then filtered our data for those trips to the subset of each trip from the last stop before entering the corridor to the first stop after leaving the corridor.
>
> Here is a list of routes included in our analysis: E13, 103, E14, 11, 33, 106, 62, 134, 109, 86, E18, E11, E15, E16, 102, 142, E10, 129, 38, 30, 51, E17, E12, 88
>
>Additionally, attached is a map of the corridor and transit routes within it.


In [55]:
corridor = gcsgp.read_file(f'{ADHOC_PATH}sccp_sacramento_2025_10_08.geojson').assign(corridor_id = 1, corridor_distance_meters = 2250)
#  use short side for distance

In [56]:
analysis_date = rt_dates.DATES['oct2025']

In [57]:
analysis_date

'2025-10-15'

In [58]:
import importlib
importlib.reload(corridor_analysis)

<module 'segment_speed_utils.corridor_analysis' from '/home/jovyan/data-analyses/rt_segment_speeds/segment_speed_utils/corridor_analysis.py'>

In [59]:
segment_speeds = corridor_analysis.import_speedmap_segment_speeds(analysis_date)
frequencies = corridor_analysis.get_max_frequencies(segment_speeds)
trip_speeds = corridor_analysis.import_trip_speeds(analysis_date)

2.5 percent of segments have no speed


In [73]:
feeds = ["Sacramento Schedule", "Elk Grove Schedule"]

In [74]:
segment_speeds = segment_speeds.query('name.isin(@feeds)')

In [75]:
corridor_trips = corridor_analysis.find_corridor_data(segment_speeds, corridor, trip_speeds,
                                                     custom_relevance_meters=100)
corridor_results = corridor_analysis.analyze_corridor_trips(corridor_trips)

1.8 percent of trips with zero seconds
0.4 percent of trips with speeds > 80mph or < 3mph


In [76]:
corridor_trips.route_short_name.unique()

array(['E13', '103', 'E14', '11', '33', '106', '62', '134', '109', '86',
       'E18', 'E11', 'E15', 'E16', '102', '142', 'E10', '129', '38', '30',
       '51', 'E17', 'E12', '88'], dtype=object)

In [77]:
', '.join(corridor_trips.route_short_name.unique())

'E13, 103, E14, 11, 33, 106, 62, 134, 109, 86, E18, E11, E15, E16, 102, 142, E10, 129, 38, 30, 51, E17, E12, 88'

In [78]:
import pandas as pd
import geopandas as gpd

from shared_utils import rt_utils, catalog_utils, rt_dates, webmap_utils

from calitp_data_analysis import get_fs, geography_utils
from segment_speed_utils import helpers, time_series_utils, gtfs_schedule_wrangling, corridor_analysis
from segment_speed_utils.project_vars import SCHED_GCS, SEGMENT_GCS, GTFS_DATA_DICT
import numpy as np
import datetime as dt

ADHOC_PATH = "gs://calitp-analytics-data/data-analyses/rt_segment_speeds/adhoc/"

In [85]:
# corridor_analysis.validate_corridor_routes(corridor, corridor_trips)

In [80]:
corridor_improvements = corridor_analysis.analyze_corridor_improvements(corridor_results, **{'trip_mph_floor': 16, 'trip_percent_speedup': 15})

median: 10.01mph... mph floor mode: 16mph


In [81]:
corridor_improvements.head(3)

,trip_instance_key,corridor_meters,corridor_seconds,corridor_speed_mps,corridor_speed_mph,route_short_name,route_id,shape_array_key,shape_id,schedule_gtfs_dataset_key,time_of_day,corridor_id,improved_corridor_speed_mph,intervention_assumption,improved_corridor_speed_mps,improved_corridor_seconds
0,000896d582b06b37fe4b501bc55fffea,1335.012451,266.0,5.018844,11.227154,86,086,b2643db097730669909d9a65882069e7,51158,5ebc3170011ae80d56831184d5035743,PM Peak,1,16.0,trips achieve 16mph or existing spd if higher,7.152436,186.651428
1,00c33cd6e4c7ef7c6e29d5f371bb17bc,3838.166749,710.0,5.405869,12.092928,11,011,f3c8087de50c58703ee97ef16d264934,51086,5ebc3170011ae80d56831184d5035743,Midday,1,16.0,trips achieve 16mph or existing spd if higher,7.152436,536.623689
2,0156358cbde57d5e9d43164e319e6c0f,1496.585551,269.0,5.563515,12.445583,86,086,b2643db097730669909d9a65882069e7,51158,5ebc3170011ae80d56831184d5035743,AM Peak,1,16.0,trips achieve 16mph or existing spd if higher,7.152436,209.241367


In [82]:
summarized = corridor_analysis.summarize_corridor_improvements(corridor_improvements, frequencies, corridor)

In [84]:
summarized.delay_minutes.sum()

2159.5